In [1]:
import numpy as np 
import pandas as pd 
import csv 
import os 

import torch 


torch.set_printoptions(edgeitems=2, precision=2, linewidth=75)

In [2]:
path = os.getcwd()
DATA_PATH = "/".join(p for p in path.split(os.path.sep)[:-1]) + "/data/p1ch4/"
CSV_PATH = DATA_PATH + "bike-sharing-dataset/hour-fixed.csv"

In [4]:
bikes_numpy = np.loadtxt(CSV_PATH, dtype=np.float32, delimiter=',', skiprows=1, 
                    converters={1: lambda x: float(x[8:10])})

bikes = torch.from_numpy(bikes_numpy)
bikes, bikes.shape

(tensor([[1.00e+00, 1.00e+00,  ..., 1.30e+01, 1.60e+01],
         [2.00e+00, 1.00e+00,  ..., 3.20e+01, 4.00e+01],
         ...,
         [1.74e+04, 3.10e+01,  ..., 4.80e+01, 6.10e+01],
         [1.74e+04, 3.10e+01,  ..., 3.70e+01, 4.90e+01]]),
 torch.Size([17520, 17]))

In [8]:
bikes.shape, bikes.shape[1], bikes.stride()

(torch.Size([17520, 17]), 17, (17, 1))

In [6]:
daily_bikes = bikes.view(-1, 24, bikes.shape[1])

daily_bikes, daily_bikes.shape

(tensor([[[1.00e+00, 1.00e+00,  ..., 1.30e+01, 1.60e+01],
          [2.00e+00, 1.00e+00,  ..., 3.20e+01, 4.00e+01],
          ...,
          [2.30e+01, 1.00e+00,  ..., 1.70e+01, 2.80e+01],
          [2.40e+01, 1.00e+00,  ..., 2.40e+01, 3.90e+01]],
 
         [[2.50e+01, 2.00e+00,  ..., 1.30e+01, 1.70e+01],
          [2.60e+01, 2.00e+00,  ..., 1.60e+01, 1.70e+01],
          ...,
          [4.60e+01, 2.00e+00,  ..., 9.00e+00, 9.00e+00],
          [4.70e+01, 2.00e+00,  ..., 8.00e+00, 8.00e+00]],
 
         ...,
 
         [[1.73e+04, 3.00e+01,  ..., 4.10e+01, 4.10e+01],
          [1.73e+04, 3.00e+01,  ..., 2.70e+01, 2.80e+01],
          ...,
          [1.74e+04, 3.00e+01,  ..., 3.00e+01, 3.60e+01],
          [1.74e+04, 3.00e+01,  ..., 3.90e+01, 4.90e+01]],
 
         [[1.74e+04, 3.10e+01,  ..., 3.00e+01, 3.40e+01],
          [1.74e+04, 3.10e+01,  ..., 1.30e+01, 1.90e+01],
          ...,
          [1.74e+04, 3.10e+01,  ..., 4.80e+01, 6.10e+01],
          [1.74e+04, 3.10e+01,  ..., 3.70e+01

In [9]:
daily_bikes = daily_bikes.transpose(1, 2)
daily_bikes.shape, daily_bikes.stride()

(torch.Size([730, 17, 24]), (408, 1, 17))

In [11]:
first_day = bikes[:24].long()
weather_onehot = torch.zeros(first_day.shape[0], 4)

first_day.shape, weather_onehot.shape

(torch.Size([24, 17]), torch.Size([24, 4]))

In [12]:
first_day[:, 9]

tensor([1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 2, 2,
        2, 2])

In [13]:
weather_onehot.scatter_(dim=1, index=first_day[:, 9].unsqueeze(1).long() - 1, value=1.0)

weather_onehot, weather_onehot.shape

(tensor([[1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [0., 1., 0., 0.],
         [1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [0., 1., 0., 0.],
         [0., 1., 0., 0.],
         [0., 1., 0., 0.],
         [0., 1., 0., 0.],
         [0., 1., 0., 0.],
         [0., 0., 1., 0.],
         [0., 0., 1., 0.],
         [0., 1., 0., 0.],
         [0., 1., 0., 0.],
         [0., 1., 0., 0.],
         [0., 1., 0., 0.]]),
 torch.Size([24, 4]))

In [21]:
torch.cat((bikes[:24], weather_onehot), dim=1)[:1]

tensor([[ 1.00,  1.00,  1.00,  0.00,  1.00,  0.00,  0.00,  6.00,  0.00,
          1.00,  0.24,  0.29,  0.81,  0.00,  3.00, 13.00, 16.00,  1.00,
          0.00,  0.00,  0.00]])

In [22]:
daily_weather_onehot = torch.zeros(daily_bikes.shape[0], 4, daily_bikes.shape[2])

daily_weather_onehot, daily_weather_onehot.shape

(tensor([[[0., 0.,  ..., 0., 0.],
          [0., 0.,  ..., 0., 0.],
          [0., 0.,  ..., 0., 0.],
          [0., 0.,  ..., 0., 0.]],
 
         [[0., 0.,  ..., 0., 0.],
          [0., 0.,  ..., 0., 0.],
          [0., 0.,  ..., 0., 0.],
          [0., 0.,  ..., 0., 0.]],
 
         ...,
 
         [[0., 0.,  ..., 0., 0.],
          [0., 0.,  ..., 0., 0.],
          [0., 0.,  ..., 0., 0.],
          [0., 0.,  ..., 0., 0.]],
 
         [[0., 0.,  ..., 0., 0.],
          [0., 0.,  ..., 0., 0.],
          [0., 0.,  ..., 0., 0.],
          [0., 0.,  ..., 0., 0.]]]),
 torch.Size([730, 4, 24]))

In [25]:
daily_weather_onehot.scatter_(dim=1, index=daily_bikes[:,9,:].long().unsqueeze(1) - 1, value=1.0)
daily_weather_onehot.shape

torch.Size([730, 4, 24])

In [27]:
daily_bikes, daily_bikes.shape

(tensor([[[1.00e+00, 2.00e+00,  ..., 2.30e+01, 2.40e+01],
          [1.00e+00, 1.00e+00,  ..., 1.00e+00, 1.00e+00],
          ...,
          [1.30e+01, 3.20e+01,  ..., 1.70e+01, 2.40e+01],
          [1.60e+01, 4.00e+01,  ..., 2.80e+01, 3.90e+01]],
 
         [[2.50e+01, 2.60e+01,  ..., 4.60e+01, 4.70e+01],
          [2.00e+00, 2.00e+00,  ..., 2.00e+00, 2.00e+00],
          ...,
          [1.30e+01, 1.60e+01,  ..., 9.00e+00, 8.00e+00],
          [1.70e+01, 1.70e+01,  ..., 9.00e+00, 8.00e+00]],
 
         ...,
 
         [[1.73e+04, 1.73e+04,  ..., 1.74e+04, 1.74e+04],
          [3.00e+01, 3.00e+01,  ..., 3.00e+01, 3.00e+01],
          ...,
          [4.10e+01, 2.70e+01,  ..., 3.00e+01, 3.90e+01],
          [4.10e+01, 2.80e+01,  ..., 3.60e+01, 4.90e+01]],
 
         [[1.74e+04, 1.74e+04,  ..., 1.74e+04, 1.74e+04],
          [3.10e+01, 3.10e+01,  ..., 3.10e+01, 3.10e+01],
          ...,
          [3.00e+01, 1.30e+01,  ..., 4.80e+01, 3.70e+01],
          [3.40e+01, 1.90e+01,  ..., 6.10e+01